# 7. Long-Term memory

Must read:
- [What is Memory?](https://langchain-ai.github.io/langgraph/concepts/memory/#what-is-memory)

This code creates a chatbot that can remember things you tell it and use those memories to give better responses. 

1) Stores what you say in Qdrant after turning your words into vectors so the bot can find related information later.
2) When you ask the bot something, it checks the database for anything related to your question. 
3) It uses this information, along with your current input, to create a reply that feels like it “remembers” past conversations. 
4) If you tell it to “remember” something, it saves both your words and its reply, so it can recall them in the future.

In [126]:
from langchain_ollama import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
import uuid
from qdrant_client.http import models
from qdrant_client.models import PointStruct
from sentence_transformers import SentenceTransformer
from qdrant_client.models import PointStruct, VectorParams
import os
OLLAMA_SERVER = os.getenv("OLLAMA_SERVER")

In [132]:
# Initialize ChatOllama and Qdrant client
llm               = ChatOllama(base_url=OLLAMA_SERVER, model="mistral", temperature=0.8)
embeddings        = OllamaEmbeddings(base_url=OLLAMA_SERVER, model="mxbai-embed-large")
qdrant_client     = QdrantClient(host='lawboxai_qdrant')
qdrant_collection = "long_term_memory"


In [133]:
class LongTermMemoryManager:
    def __init__(self, collection_name, qdrant_client, embeddings):
        self.collection_name = collection_name
        self.qdrant_client = qdrant_client
        self.embeddings = embeddings

        # Create collection if it doesn't exist
        if collection_name not in [c.name for c in qdrant_client.get_collections().collections]:
            self.qdrant_client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(size=len(self.embeddings.embed_query("test")),distance="Cosine"))

    def store_memory(self, content):
        self.qdrant_client.upsert(
            collection_name=self.collection_name,
            points=[PointStruct(
                id=str(uuid.uuid5(uuid.NAMESPACE_DNS, content)),
                vector=self.embeddings.embed_query(content),
                payload={"content": content}
            )]
        )

    def retrieve_memory(self, query, top_k=3):
        results = self.qdrant_client.query_points(
            collection_name=self.collection_name,
            query=self.embeddings.embed_query(query),
            limit=top_k
        )
        return [point.payload["content"] for point in results.points]

In [134]:
collection_name = "long_term_memory"
memory_manager = LongTermMemoryManager(collection_name, qdrant_client, embeddings)


In [135]:
def chat_with_memory(user_input):
    # Retrieve relevant memories
    relevant_memories = memory_manager.retrieve_memory(user_input)
    system_message = {
        "role": "system",
        "content": "Please respond in French, and incorporate any relevant context from the following: " + " ".join(relevant_memories)
    }
    prompt = [
        system_message,
        {"role": "user", "content": user_input}
    ]
    response = llm.invoke(prompt)
    ai_reply = response.content.strip()
    if "remember" in user_input.lower():
        memory_manager.store_memory(user_input)
        memory_manager.store_memory(ai_reply)
    
    return ai_reply

In [136]:
interactions = [
    "I told you something about Paris. Can you recall?",
    "Do you remember what I told you about myself?",
    "Tell me something about my name and where I live.",
    "Hello, my name is Alex.",
    "Can you remember my name?",
    "Remember that I live in Paris.",
    "Where do I live?",
    "What is the capital of France?",
    "Can you summarize what we discussed earlier?",
    "Goodbye, and remember my details for next time."
]

for i, user_input in enumerate(interactions, 1):
    print(f"Interaction {i}")
    print(f"You: {user_input}")
    reply = chat_with_memory(user_input)
    print(f"Assistant: {reply}")
    print("-" * 30)

Interaction 1
You: I told you something about Paris. Can you recall?
Assistant: Bonjour, oui, je me souviens bien que tu vas vivre à Paris ! Tu m'as dit précédemment qu'il est ta ville d'origine ou où tu résides actuellement. C'est une superbe ville avec de nombreuses attractions, comme la Tour Eiffel, le Louvre et Notre-Dame. Je t'espère que tu apprécies la vie à Paris !
------------------------------
Interaction 2
You: Do you remember what I told you about myself?
Assistant: Oui, je me souviens de ce que tu m'as dit à propos de toi-même. Tu t'appelles... (mentionner le nom)

Au revoir et ne oublie pas mes détails pour la prochaine fois.
------------------------------
Interaction 3
You: Tell me something about my name and where I live.
Assistant: Oui, je te rappelle ton nom, c'est [Ton Nom]. Tu habites à Paris, une ville merveilleuse et historique. En ce qui concerne ton nom, il est important de savoir que "Nom" en français est un mot très commun et utilisé pour désigner le nom d'une 